In [2]:
from __future__ import unicode_literals # 이게 뭘까
import requests
import pymongo
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
# from scrapy.http import TextResponse
import urllib.request
import urllib
import getpass
import time
import re
# from google.cloud import vision
# from google.cloud.vision import types
import os
import io
from os import walk
from bs4 import BeautifulSoup

In [10]:
!cd

C:\Users\Amps\Desktop\Project\Project03_Data_Science


In [12]:
URL = 'https://page.kakao.com/home?seriesId=57563035'

driver = webdriver.Chrome(executable_path='C:\chromedriver')
driver.implicitly_wait(time_to_wait=1)
driver.get(URL)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

title = soup.find('h2', {'class': 'text-ellipsis css-jgjrt'}).get_text() # 제목 수집
author = soup.find_all('div', {'class': 'text-ellipsis css-7a7cma'})[1].get_text() # 작가 수집

print(title, author)

driver.execute_script("window.scrollTo(0, 2000);")

In [50]:
data_list=soup.findAll('ul',{'class':'css-14gr98z'})

li_list = []
for data in data_list:
    li_list.extend(data.findAll('li')) #해당 부분을 찾아 li_list와 병합
# pprint(li_list)

#각각 썸네일과 제목 추출하기
thumb = []
for li in li_list:
    img = li.find('img')
    img_src = img['data-src']
    print('https:' + img_src)
    thumb.append('https:' + img_src)

In [ ]:
import os
import time
import urllib.request

for i in range(len(thumb)):
    urllib.request.urlretrieve(thumb[i], './img/' + title + '_' +str(i+1) +".jpg")

In [3]:
webtoon_info = []
total_df_info = []
total_img_count = 0

domain_url = 'https://page.kakao.com/main'
main_url = 'https://page.kakao.com/main?categoryUid=10&subCategoryUid=1002'

html = requests.get(main_url).text
soup = BeautifulSoup(html, 'html.parser')
webtoon_area = soup.select('#root > div.jsx-3157985592.mainContents.mainContents_pc > div.css-1sna24c > div.css-1saqd06 > div')

for each_webtoon in webtoon_area:
    webtoon_last_page = domain_url + each_webtoon['href'] + '&page={}'.format(99999)
    html = requests.get(webtoon_last_page).text
    soup = BeautifulSoup(html, 'html.parser')
    last_page_num = int(soup.select('#content > div.paginate > div > strong.page')[0].find('em').text)

    title = ', '.join([each_ele.strip() for each_ele in soup.select('.comicinfo h2')[0].text.split('\n') if each_ele != ''])
    title = ', '.join(['&'.join([each_text.strip() for each_text in each_text.split('/')]) for each_text in title.split(',')])
    title = re.sub('[-=+#/\?:^$.@*\"※~%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', title)

    genre = soup.select('#content > div.comicinfo > div.detail > p.detail_info > span.genre')[0].text
    content = soup.select('#content > div.comicinfo > div.detail > p:nth-child(2)')[0].text
    age = soup.select('#content > div.comicinfo > div.detail > p.detail_info > span.age')[0].text

    webtoon_info.append({'title': title, 'genre': genre, 'content': content, 'age': age})

    for page_num in range(1, last_page_num + 1):
        webtoon_url = domain_url + each_webtoon['href'] + '&page={}'.format(page_num)
        html = requests.get(webtoon_url).text
        soup = BeautifulSoup(html, 'html.parser')

        if not os.path.exists('./imgs/{}'.format(title)):
            os.mkdir('./imgs/{}'.format(title))

        thumbnail_area = soup.find('table', 'viewList')
        thumbnail_img_list = thumbnail_area.select('td > a > img')[1:]
        
#         for each_img_tag in thumbnail_img_list:
#             print(re.sub('[^0-9a-zA-Zㄱ-힗]','',each_img_tag['alt'].split('/')[0]))

        for each_img_tag in thumbnail_img_list:
            urllib.request.urlretrieve(each_img_tag['src'], './imgs/{}/{}.png'.format(title, title + '_' + re.sub('[^0-9a-zA-Zㄱ-힗]','',str(each_img_tag['alt'].split('/')[0]))))
#             urllib.request.urlretrieve(each_img_tag['src'], './total_img/{}.png'.format(title + '_' + '&'.join(each_img_tag['alt'].split('/')[0])))
#             total_df_info.append({'title': title + '_' + '&'.join(each_img_tag['alt'].split('/')), 'genre': genre, 'content': content, 'age': age})
            total_img_count += 1

# df = pd.DataFrame(webtoon_info, columns=['title', 'genre', 'content', 'age'])
# total_df = pd.DataFrame(total_df_info, columns=['title', 'genre', 'content', 'age'])
# df.to_csv('webtoon_info.csv')
# total_df.to_csv('total_img_info.csv')

KeyError: 'href'